# Aries Basic Controller - Holder Bob

This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [issuer notebook](http://localhost:8888/notebooks/issuer.ipynb) from Alice's perspective.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

## Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials", payload)
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Handle Credentials {'trace': False, 'created_at': '2020-07-09 10:33:52.686956Z', 'auto_remove': True, 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'initiator': 'external', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '737fa367-1fe6-4633-ba1d-ad4822b5751d', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'comment': 'create automated credential exchange'}, 'auto_issue': False, 'thread_id': 'ae10443d-767f-4989-bce8-62237b3c847c', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'connection_id': '7748b251-47ef-495b-892f-4eb8b3216c38', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4

## Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.

In [13]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 9fa74567-17a5-401a-a336-a8cef829c255, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Propose Credential

If unhappy with the attributes being offered (e.g. there is a mistake), the holder can send a proposal to the issuer for an alternative credential. The holder can also initiate the protocol with a proposal to the issuer, this is left out of scope in the tutorial, but should be possible to edit to achieve this if interested.

Note the proposal will create a new credential exchange record.

**Arguments (These can all be found in a credential exchange record)**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [24]:
print(results[0]['schema_id'])
connection_id = cred_record['connection_id']
schema_id = cred_record['schema_id']
cred_def_id = cred_record['credential_definition_id']
proposed_attributes = [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]

proposal_record = await agent_controller.issuer.send_proposal(connection_id, schema_id, cred_def_id, proposed_attributes)
proposal_record_id = record[credential_exchange_id]
proposal_state = record['state']
proposal_role = record['role']
proposal_attributes = record['credential_proposal_dict']['credential_proposal']['attributes']
print(f"Proposal credential exchange record {proposal_record_id}, role: {proposal_role}, state: {proposal_state}")
print(f"Attributes: {attributes}")


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1
{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]}, 'connection_id': '7748b251-47ef-495b-892f-4eb8b3216c38', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'schema_name': 'open_mined_contributor', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Handle Credentials {'trace': False, 'created_at': '2020-07-09 11:11:12.146394Z', 'auto_remove': True, 'initiator': 'self', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'a40785bf-5fe2-4885-b43e-a307840b8767', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contr

## Request Credential from Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [30]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials {'trace': False, 'created_at': '2020-07-09 10:40:29.429869Z', 'auto_remove': True, 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'initiator': 'external', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'c06e4be3-0b89-44b9-a33c-a3ae6802e4b7', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'comment': 'create automated credential exchange'}, 'auto_issue': False, 'credential_request': {'prover_did': 'YHF8W9oKsnCmp3qKtsV1As', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'blinded_ms': {'u': '629175162381331638193017546051786149049205084598064170958622038561907755408632568347266832640

Handle Credentials {'trace': False, 'created_at': '2020-07-09 10:40:29.429869Z', 'auto_remove': True, 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'initiator': 'external', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'c06e4be3-0b89-44b9-a33c-a3ae6802e4b7', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'comment': 'create automated credential exchange'}, 'auto_issue': False, 'credential_request': {'prover_did': 'YHF8W9oKsnCmp3qKtsV1As', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'blinded_ms': {'u': '629175162381331638193017546051786149049205084598064170958622038561907755408632568347266832640

## Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [32]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 9fa74567-17a5-401a-a336-a8cef829c255, role: holder, state: credential_received


In [34]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials {'credential_id': 'My OM Credential', 'trace': False, 'created_at': '2020-07-09 10:40:29.429869Z', 'auto_remove': True, 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'initiator': 'external', 'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'c06e4be3-0b89-44b9-a33c-a3ae6802e4b7', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'comment': 'create automated credential exchange'}, 'auto_issue': False, 'credential_request': {'prover_did': 'YHF8W9oKsnCmp3qKtsV1As', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'blinded_ms': {'u': '62917516238133163819301754605178614904920508459806417095

In [35]:
## End of Tutorial


Be sure to terminate the controller so you can run another tutorial.

ClientResponseError: 404, message='Not Found', url=URL('http://bob-agent:8051/issue-credential/records/9fa74567-17a5-401a-a336-a8cef829c255')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site-packages/